## Preparations

In [43]:
import pandas as pd
import math

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

px.defaults.template = "plotly_white"

## Import and clean data

### Weather

In [44]:
weather_data_file = 'data/Elsenburg_hourly_20230101_20231231.csv'

# Read the CSV file into a DataFrame
df_weather = pd.read_csv(weather_data_file)
df_weather.sample(3).T

,4460,5710,6023
LogDate,2023-07-05T00:00:00,2023-08-26T00:00:00,2023-09-08T00:00:00
LogTime,21:00,23:00,24:00
WindSpeed,0.0,0.0,0.14
Temp,12.05,7.52,10.45
Humidity,68.9,88.0,96.1
Rainfall,0.0,0.0,0.0
Radiation,0.0,0.0,0.0
DataType,Logger,Logger,Logger
WindDegrees,224,190,345
SHrs,0.0,0.0,0.0


Date
- LogDate > deleted
- LogTime > deleted
- LogDateTime > converted to datetime index

Temperature
- Temp > OK

Humidity
- Humidity > OK
- Rainfall > OK

Radiation
- Radiation > deleted (taken from the PV-GIS data)
- DataType > deleted
- SHrs > deleted

Wind
- WindSpeed > OK
- WindDirection > deleted
- WindDegrees > OK

Others
- PenmanEto > deleted

In [45]:
drop_cols = ['LogDate', 'LogTime', 'LogDateTime', 'Radiation', 'DataType', 'SHrs', 'PenmanEto', 'WindDirection']

In [46]:
# Read the CSV file into a DataFrame
df_weather = pd.read_csv(weather_data_file)

# Convert 'LogDateTime' to datetime format and set as index
df_weather['t'] = pd.to_datetime(df_weather['LogDateTime'])
df_weather.set_index('t', inplace=True)

# Drop unnecessary columns
df_weather.drop(columns=drop_cols, inplace=True)

# Sort the columns manually
df_weather = df_weather[['Temp', 'Humidity', 'WindSpeed', 'WindDegrees', 'Rainfall']]

# Round the values to one decimal place
df_weather = df_weather.round(1)

# Rename the columns
df_weather.rename(columns={'Temp': 'AirTemp', 'Humidity': 'RH'}, inplace=True)

df_weather

,AirTemp,RH,WindSpeed,WindDegrees,Rainfall
t,,,,,
2023-01-01 01:00:00,21.0,62.6,0.4,273,0.0
2023-01-01 02:00:00,19.0,75.1,0.3,299,0.0
2023-01-01 03:00:00,17.9,81.0,0.5,304,0.0
2023-01-01 04:00:00,17.2,84.3,0.1,263,0.0
2023-01-01 05:00:00,17.0,83.9,0.2,267,0.0
...,...,...,...,...,...
2023-12-31 19:00:00,26.8,56.5,3.0,112,0.0
2023-12-31 20:00:00,25.9,57.5,3.0,111,0.0
2023-12-31 21:00:00,24.9,58.4,1.4,101,0.0


### PV

In [47]:
pv_data_file = 'data/Elsenburg_-33.842_18.840_SA3_1kWp_crystSi_14_-1deg_-180deg_2023_2023.csv'

# Read the PV data CSV file into a DataFrame, skipping the first 10 and last 10 rows
df_pv = pd.read_csv(pv_data_file, skiprows=10, skipfooter=10, engine='python')
df_pv

,time,P,G(i),H_sun,T2m,WS10m,Int
0,20230101:0002,0.00,0.0,0.0,17.07,2.83,0.0
1,20230101:0102,0.00,0.0,0.0,16.51,2.69,0.0
2,20230101:0202,0.00,0.0,0.0,16.04,2.69,0.0
3,20230101:0302,0.00,0.0,0.0,15.70,2.69,0.0
4,20230101:0402,9.49,23.0,3.8,15.41,2.55,0.0
...,...,...,...,...,...,...,...
8755,20231231:1902,0.00,0.0,0.0,21.33,4.41,0.0
8756,20231231:2002,0.00,0.0,0.0,20.47,4.34,0.0
8757,20231231:2102,0.00,0.0,0.0,19.68,4.00,0.0
8758,20231231:2202,0.00,0.0,0.0,19.17,4.00,0.0


Installed peak PV power = 1 kWp

- time
- P: PV system power (W)
- G(i): Global irradiance on the inclined plane (plane of the array) (W/m2)
- H_sun: Sun height (degree)
- T2m: 2-m air temperature (degree Celsius)
- WS10m: 10-m total wind speed (m/s)
- Int: 1 means solar radiation values are reconstructed

In [48]:
drop_cols = ['time', 'G(i)', 'H_sun', 'Int']

In [49]:
# Read the data from csv
df_pv = pd.read_csv(pv_data_file, skiprows=10, skipfooter=10, engine='python')

# Convert the column time to datetime and set it as index
df_pv['t'] = pd.to_datetime(df_pv['time'], format='%Y%m%d:%H%M', errors='coerce')
df_pv.set_index('t', inplace=True)

# Remove the minutes (round down to the nearest hour)
df_pv.index = df_pv.index.floor('h') # type: ignore

# Rename the columns
df_pv.rename({'P': 'PV_P', 'T2m': 'PV_AirTemp', 'WS10m': 'PV_WindSpeed'}, axis=1, inplace=True)

# Drop unnecessary columns
df_pv.drop(columns=drop_cols, inplace=True)

# Round the values to one decimal place
df_pv = df_pv.round(1)

df_pv

,PV_P,PV_AirTemp,PV_WindSpeed
t,,,
2023-01-01 00:00:00,0.0,17.1,2.8
2023-01-01 01:00:00,0.0,16.5,2.7
2023-01-01 02:00:00,0.0,16.0,2.7
2023-01-01 03:00:00,0.0,15.7,2.7
2023-01-01 04:00:00,9.5,15.4,2.6
...,...,...,...
2023-12-31 19:00:00,0.0,21.3,4.4
2023-12-31 20:00:00,0.0,20.5,4.3
2023-12-31 21:00:00,0.0,19.7,4.0


### Combine the data

In [50]:
df_weather

,AirTemp,RH,WindSpeed,WindDegrees,Rainfall
t,,,,,
2023-01-01 01:00:00,21.0,62.6,0.4,273,0.0
2023-01-01 02:00:00,19.0,75.1,0.3,299,0.0
2023-01-01 03:00:00,17.9,81.0,0.5,304,0.0
2023-01-01 04:00:00,17.2,84.3,0.1,263,0.0
2023-01-01 05:00:00,17.0,83.9,0.2,267,0.0
...,...,...,...,...,...
2023-12-31 19:00:00,26.8,56.5,3.0,112,0.0
2023-12-31 20:00:00,25.9,57.5,3.0,111,0.0
2023-12-31 21:00:00,24.9,58.4,1.4,101,0.0


In [51]:
df_pv

,PV_P,PV_AirTemp,PV_WindSpeed
t,,,
2023-01-01 00:00:00,0.0,17.1,2.8
2023-01-01 01:00:00,0.0,16.5,2.7
2023-01-01 02:00:00,0.0,16.0,2.7
2023-01-01 03:00:00,0.0,15.7,2.7
2023-01-01 04:00:00,9.5,15.4,2.6
...,...,...,...
2023-12-31 19:00:00,0.0,21.3,4.4
2023-12-31 20:00:00,0.0,20.5,4.3
2023-12-31 21:00:00,0.0,19.7,4.0


In [52]:
# Merge the two DataFrames on their datetime index
df = pd.merge(df_pv, df_weather, left_index=True, right_index=True)
df.head()

,PV_P,PV_AirTemp,PV_WindSpeed,AirTemp,RH,WindSpeed,WindDegrees,Rainfall
t,,,,,,,,
2023-01-01 01:00:00,0.0,16.5,2.7,21.0,62.6,0.4,273,0.0
2023-01-01 02:00:00,0.0,16.0,2.7,19.0,75.1,0.3,299,0.0
2023-01-01 03:00:00,0.0,15.7,2.7,17.9,81.0,0.5,304,0.0
2023-01-01 04:00:00,9.5,15.4,2.6,17.2,84.3,0.1,263,0.0
2023-01-01 05:00:00,168.9,17.1,2.6,17.0,83.9,0.2,267,0.0


In [53]:
# Create a scatter plot comparing Temperatures
px.scatter(df, x='PV_AirTemp', y='AirTemp', opacity=0.25, width=600, height=600)

In [54]:
# Create a scatter plot comparing Wind Speeds
px.scatter(df, x='PV_WindSpeed', y='WindSpeed', opacity=0.25, width=600, height=600)

In [55]:
# Export the data as xlsx-file
df.to_excel('data/Elsenburg.xlsx')

## Calculations

In [56]:
def calculate_dew_point(T_air_c, RH_percent):
    """
    Calculates the dew point in Celsius using the August-Roche-Magnus formula.
    
    Parameters:
    T_air_c (float): Current air temperature in degrees Celsius.
    RH_percent (float): Current relative humidity in percent (e.g., 70.0).
    
    Returns:
    float: The dew point temperature in degrees Celsius.
    """
    
    # Constants for the August-Roche-Magnus formula
    b = 17.62
    c = 243.12
    
    # Convert humidity from percent to a fraction
    RH_fraction = RH_percent / 100.0
    
    # Calculate the intermediate gamma value
    gamma = math.log(RH_fraction) + (b * T_air_c) / (c + T_air_c)
    
    # Calculate the dew point
    T_dew_point_c = (c * gamma) / (b - gamma)
    
    return round(T_dew_point_c, 1)

# Test the function
print(f'Calculated dew point = {calculate_dew_point(17.1, 81.1)} °C')

Calculated dew point = 13.8 °C


In [57]:
# Create a new column 'DewPoint' by applying the calculate_dew_point function row-wise
df['DewPoint'] = df.apply(lambda r: calculate_dew_point(r['AirTemp'], r['RH']), axis=1)

# Calculate the ...
df['CoolingTemp'] = df['DewPoint'] - df['AirTemp']

In [70]:
# Create subplots with shared x-axis and add a centered main title
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0)
fig.update_layout(title_text='PV Power (1 kWp) and Cooling Temp to reach the Dew Point', title_x=0.5)

# Add trace for PV Power
fig.add_trace(
    go.Scatter(x=df.index, y=df['PV_P'], mode='lines', name='PV Power (W)',
               line=dict(color='orange')),
    row=1, col=1
)

# Add trace for Cooling Temperature
fig.add_trace(
    go.Scatter(x=df.index, y=df['CoolingTemp'], mode='lines', name='CoolingTemp (°C)',
               line=dict(color='royalblue')),
    row=2, col=1
)

# Update axis titles and layout
fig.update_yaxes(title_text='PV Power / W', row=1, col=1)
fig.update_yaxes(title_text='CoolingTemp / K', row=2, col=1)
fig.update_xaxes(matches='x')  # ensure x-axes are synced when zooming
fig.update_layout(height=600, width=900, template=px.defaults.template, showlegend=False)

fig.write_html('pv_power_cooling_temp.html')
fig.show()

In [59]:
# Export the data as xlsx-file
df.to_excel('data/Elsenburg.xlsx')